# Multi-Agent Orchestration with Semantic Kernel

Welcome to the fascinating world of multi-agent orchestration! In this notebook, we'll explore how multiple AI agents can work together to solve complex problems through different orchestration patterns.



## Package References and Configuration

First, let's install the required NuGet packages for multi-agent orchestration and create our kernel

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.61.0"

#!import config/Settings.cs

using Microsoft.SemanticKernel;
using Kernel = Microsoft.SemanticKernel.Kernel;

Kernel CreateKernel()
{
    var builder = Kernel.CreateBuilder();

    var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId, embeddingEndpoint, embeddingApiKey) = Settings.LoadFromFile();

    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
    var kernel = builder.Build();

    return kernel;
}

var kernel = CreateKernel();

Installed Packages Microsoft.SemanticKernel, 1.61.0

## Configuration and Settings

Load our AI configuration settings:

## Using Statements and Dependencies

Import all necessary namespaces for multi-agent orchestration:

In [2]:
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.61.0"
#r "nuget: Microsoft.SemanticKernel.Agents.Orchestration, 1.61.0-preview"
#r "nuget: Microsoft.SemanticKernel.Agents.Runtime.InProcess, 1.61.0-preview"

Installed Packages Microsoft.SemanticKernel.Agents.Core, 1.61.0 Microsoft.SemanticKernel.Agents.Orchestration, 1.61.0-preview Microsoft.SemanticKernel.Agents.Runtime.InProcess, 1.61.0-preview

because we are using a preview and (still) experimental version, we need to diable the warning:
*error SKEXP0110: 'Microsoft.SemanticKernel.Agents.Orchestration.Concurrent.ConcurrentOrchestration' is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.*

In [3]:
#pragma warning disable SKEXP0110

In [4]:
// Core Semantic Kernel imports
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

// System imports for orchestration
using System;
using System.Collections.Generic;
using System.Threading.Tasks;
using System.Threading;
using System.Linq;
using System.Collections.Concurrent;
using System.Text;
using Microsoft.DotNet.Interactive;

// Kernel alias to avoid conflicts
using Kernel = Microsoft.SemanticKernel.Kernel;

Console.WriteLine("All dependencies imported successfully! 🎯");

All dependencies imported successfully! 🎯


## Helper functions



In [5]:
// Helper function to display agent responses
void DisplayResponse(string agentName, string response, string emoji = "🤖")
{
    Console.WriteLine($"\n{emoji} **{agentName}:**");
    Console.WriteLine($"   {response}");
    Console.WriteLine(new string('-', 50));
}

// Helper function to display section headers
void DisplaySection(string title, string emoji = "🎯")
{
    Console.WriteLine($"\n{emoji} {title}");
    Console.WriteLine(new string('=', title.Length + 4));
}

Console.WriteLine("🛠️ Helper functions created successfully!");

🛠️ Helper functions created successfully!


## Simple Concurrent Orchestration

Let's start with our first orchestration pattern: **Concurrent Orchestration**

### Scenario: Restaurant Kitchen Team 🍽️

Imagine a busy restaurant kitchen where multiple chefs work simultaneously on different parts of a meal:

In [6]:
#pragma warning disable SKEXP0110

using Microsoft.SemanticKernel.Agents.Orchestration;
using Microsoft.SemanticKernel.Agents.Orchestration.Concurrent;
using Microsoft.SemanticKernel.Agents.Runtime.InProcess;


DisplaySection("CONCURRENT ORCHESTRATION - Restaurant Kitchen Team", "🍽️");

// Create our restaurant kitchen team
var sousChef = new ChatCompletionAgent
    {
        Name = "SousChef",
        Description = "You are a sous chef specializing in appetizers and salads",
        Instructions = "Be creative and describe the dish preparation briefly",
        Kernel = kernel,
        Arguments = new KernelArguments(new OpenAIPromptExecutionSettings
        {
            MaxTokens = 500,
            Temperature = 0.7
        })
    };

var mainChef = new ChatCompletionAgent
    {
        Name = "MainChef",
        Description = "You are a main course chef specializing in proteins and hearty dishes",
        Instructions = "Be creative and describe the cooking process briefly",
        Kernel = kernel
    };


var pastryChef = new ChatCompletionAgent
    {
        Name = "PastryChef",
        Description = "You are a pastry chef specializing in desserts and sweet treats",
        Instructions = "Be creative and describe the dessert preparation briefly",
        Kernel = kernel
    };
    
// Define the orchestration
var orchestration =
    new ConcurrentOrchestration(sousChef, mainChef, pastryChef);

// Start the runtime
var runtime = new InProcessRuntime();

await runtime.StartAsync();


OrchestrationResult<string[]> result = await orchestration.InvokeAsync("Create a delicious israeli meal", runtime);

string[] output = await result.GetValueAsync();
Console.WriteLine($"\n# RESULT:\n{string.Join("\n\n", output.Select(text => $"{text}"))}");

await runtime.RunUntilIdleAsync();


🍽️ CONCURRENT ORCHESTRATION - Restaurant Kitchen Team

# RESULT:
Let's create a vibrant Israeli meal that showcases the bold flavors, fresh ingredients, and culinary traditions of the region. Here's a delicious menu idea:

---

**Starter: Traditional Israeli Salad**
Dice fresh cucumbers, tomatoes, red onion, and parsley into small pieces. Drizzle with olive oil, fresh lemon juice, salt, and pepper. Toss gently. Optional: Add crumbled feta or za'atar for an extra Mediterranean flair.

**Main Dish: Chicken Shawarma Pita**
Marinate chicken thighs in a mixture of olive oil, lemon juice, smoked paprika, turmeric, cumin, garlic, and a pinch of cinnamon. Grill or pan-sear until golden and tender. Serve inside fluffy pita bread with tahini sauce, pickled vegetables, and a dollop of creamy hummus. Add fresh herbs and sprinkle with sumac for brightness.

**Side Dish: Golden Couscous Pilaf**
Cook fluffy couscous in vegetable stock with a touch of turmeric for color. Toss with roasted almonds, dr

### Adding the inner interaction and handling messages

In [7]:
#pragma warning disable SKEXP0110
#pragma warning disable SKEXP0001

var orchestrationWithInnerCommunication =
    new ConcurrentOrchestration(sousChef, mainChef, pastryChef)
    {
        ResponseCallback = (ChatMessageContent response)=>
            {   
                DisplayResponse(response.AuthorName, response.Content);
                return ValueTask.CompletedTask;
            }
    };

// Start the runtime
var runtime = new InProcessRuntime();

await runtime.StartAsync();


OrchestrationResult<string[]> result = await orchestrationWithInnerCommunication.InvokeAsync("Create a delicious israeli meal", runtime);

string[] output = await result.GetValueAsync();
//Console.WriteLine($"\n# RESULT:\n{string.Join("\n\n", output.Select(text => $"{text}"))}");

await runtime.RunUntilIdleAsync();




🤖 **SousChef:**
   **Dish Name:** **Sabich Platter with Fresh Israeli Salad and Tahini Drizzle**

**Preparation:**

1. **Eggplant Magic:** Slice a large eggplant into thick rounds, sprinkle with salt, and let it sweat for 20 minutes. Pat dry, brush with olive oil, and roast until golden and tender.

2. **Perfectly Boiled Eggs:** Cook eggs until hard-boiled, then peel and slice them into halves for that creamy yolk goodness.

3. **Pita Pockets:** Lightly warm fresh pita bread until soft and pillowy—ready to cradle your delicious fillings.

4. **Israeli Salad:** Dice cucumbers, ripe tomatoes, and red onion into tiny cubes. Toss with fresh parsley, lemon juice, olive oil, salt, and a hint of sumac for tangy brightness.

5. **Tahini Drizzle:** Whisk together tahini, fresh lemon juice, garlic, a pinch of cumin, and ice-cold water until it’s silky smooth and pourable.

6. **Build the Platter:** Arrange roasted eggplant, sliced eggs, and salad on a plate. Add a generous scoop of creamy hummu

## Sequential Orchestration

Now let's explore **Sequential Orchestration** where agents work in a specific order.

### Scenario: News Editorial Team 📰

In a newsroom, articles go through a specific workflow: Writing → Editing

In [8]:
#pragma warning disable SKEXP0110
#pragma warning disable SKEXP0001

using Microsoft.SemanticKernel.Agents.Orchestration.Sequential;

DisplaySection("SEQUENTIAL ORCHESTRATION - News Editorial Team", "📰");

// Create our news editorial team
var writer = new ChatCompletionAgent
    {
        Name = "Writer",
        Description = "You are a news writer",
        Instructions = "Take the research and write a compelling news article. Keep it engaging and informative",
        Kernel = kernel
    };


var editor = new ChatCompletionAgent
    {
        Name = "Editor",
        Description = "You are a news editor",
        Instructions = 
        """
        Review the given draft article and imporve clarity, correct grammer and shorten it.
        Output the final improved copy as a single text block.
        """,
        Kernel = kernel
    };
    
// Define the orchestration
var orchestration =
    new SequentialOrchestration(writer, editor)
    {
        ResponseCallback = (ChatMessageContent response)=>
            {   
                DisplayResponse(response.AuthorName, response.Content);
                return ValueTask.CompletedTask;
            }
    };

// Start the runtime
var runtime = new InProcessRuntime();

await runtime.StartAsync();


OrchestrationResult<string> result = await orchestration.InvokeAsync("Write an article on Semantic Kernel Agents", runtime);

string output = await result.GetValueAsync();
Console.WriteLine($"\n# RESULT:\n{output}");

await runtime.RunUntilIdleAsync();


📰 SEQUENTIAL ORCHESTRATION - News Editorial Team

🤖 **Writer:**
   **Revolutionizing AI Automation: The Rise of Semantic Kernel Agents**

In the rapidly accelerating domain of artificial intelligence, the term "Semantic Kernel Agents" is making waves as an emerging paradigm, signaling a transformative leap forward in automation, adaptability, and cognitive reasoning within AI systems. These sophisticated agents are empowering developers and businesses to unlock new possibilities in human-computer interaction, workflow optimization, and problem-solving, all while bridging the gap between machine learning models and real-world applications.

### What Are Semantic Kernel Agents?

Semantic Kernel Agents (SK Agents) are intelligent AI systems that leverage the power of semantic understanding—a core function in how machines interpret and process human language. At their foundation, SK Agents utilize Microsoft's Semantic Kernel, an open-source framework designed for integrating advanced larg

## Group Chat Orchestration

Let's explore **Group Chat Orchestration** where agents collaborate in a discussion.

### Scenario: Medical Consultation Team 🏥

A medical consultation where specialists discuss a patient case together:

In [9]:
DisplaySection("GROUP CHAT ORCHESTRATION - Medical Consultation Team", "🏥");

// Create our medical consultation team
var generalDoctor = new ChatCompletionAgent
    {
        Name = "GeneralDoctor",
        Description = "You are a general practitioner",
        Instructions = 
        """
        Provide overall medical assessment and coordinate with specialists. 
        Be professional and concise.
        You are part of a team of doctors and you need to take their input and evaluate your answer accordingly, so always summarize the input you received.
        """,
        Kernel = kernel
    };

 
var cardiologist = new ChatCompletionAgent
    {
        Name = "Cardiologist",
        Description = "You are a heart specialist",
        Instructions = 
        """
        Focus on cardiovascular aspects of the case. 
        Provide expert cardiac insights.
        You are part of a team of doctors and you need to take their input and evaluate your answer accordingly, so always summarize the input you received.
        """,
        Kernel = kernel
    };
    
var neurologist = new ChatCompletionAgent
    {
        Name = "Nutritionist",
        Description = "You are a neurologist",
        Instructions = 
        """
        Focus on neurological aspects of the case. 
        Provide expert brain and nervous system insights.
        You are part of a team of doctors and you need to take their input and evaluate your answer accordingly, so always summarize the input you received.
        """,
        Kernel = kernel
    };



// Create a shared chat history for the group consultation
var groupChatHistory = new ChatHistory();



🏥 GROUP CHAT ORCHESTRATION - Medical Consultation Team


In [10]:
#pragma warning disable SKEXP0110
#pragma warning disable SKEXP0001

using Microsoft.SemanticKernel.Agents.Orchestration.GroupChat;

var orchestration =
    new GroupChatOrchestration(
        new RoundRobinGroupChatManager()  { MaximumInvocationCount = 5  },
        generalDoctor,
        cardiologist,
        neurologist)
    {       
        ResponseCallback =  chatMessage => 
        { 
            groupChatHistory.Add(chatMessage);
            return ValueTask.CompletedTask;
        }
    };

// Start the runtime
var runtime = new InProcessRuntime();
await runtime.StartAsync();


OrchestrationResult<string> result = await orchestration.InvokeAsync("Everytime i workout my breath makes funny sounds. Tell me what to do", runtime);
string text = await result.GetValueAsync();
Console.WriteLine($"\n# RESULT: {text}");

await runtime.RunUntilIdleAsync();

Console.WriteLine("\n\nORCHESTRATION HISTORY");
foreach (ChatMessageContent message in groupChatHistory)
{
    DisplayResponse(message.AuthorName, message.Content);
}


# RESULT: 


ORCHESTRATION HISTORY

🤖 **GeneralDoctor:**
   Thank you for sharing your concern. Based on your description, it is possible that your symptoms are related to exercise-induced bronchoconstriction (EIB), which can cause wheezing or "funny sounds" during workouts. Here is a general medical assessment:

### Potential Causes:
1. **Exercise-Induced Bronchoconstriction (EIB)**: Common in people with asthma or without a formal asthma diagnosis, especially during or after exercise.
2. **Upper Airway Issues**: Such as vocal cord dysfunction (VCD), which can mimic asthma symptoms.
3. **Sinus or Respiratory Conditions**: Chronic allergies, sinus infections, or postnasal drip could contribute to altered breathing sounds.
4. **Cardiopulmonary Concerns**: Rarely, abnormal breath sounds during exertion could indicate cardiac or pulmonary limitations.

### Recommended Steps:
1. **Schedule an Appointment**:
   - Consult your primary care physician or a pulmonologist for a detailed evaluat

## Custom Orchestration

Finally, let's explore **Custom Orchestration** - controlling the agents selection and termincation logic.

### Scenario: Hospital's Tumor Board 🎵

Hospital's tumor board is reviewing a complex cancer case:

In [11]:
#pragma warning disable SKEXP0110
#pragma warning disable SKEXP0001

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents.Orchestration.GroupChat;
using Microsoft.SemanticKernel.ChatCompletion;

class TumorBoardLeader : GroupChatManager
{
    string _boardCase;
    IChatCompletionService _chatCompletion;

    public TumorBoardLeader(string boardCase, IChatCompletionService chatCompletion)
    {
        _boardCase = boardCase;
        _chatCompletion = chatCompletion;
    }

    private static class Prompts
    {
        public static string Termination(string boardCase) =>
            $"""
                You are a leader of the hospital tumor board. The case the board is working on is : '{boardCase}'. 
                You need to determine if the discussion has reached a conclusion and if all next steps were listed.
                if no artifacts were produced yet and you don't have knowledge of them yet, then continue the dicsussion and planning.
                
                If you would like to end the discussion, please respond with True. Otherwise, respond with False.
                """;

        public static string Selection(string boardCase, string participants) =>
            $"""
                Tou are a leader of the hospital tumor board. The case the board is working on is : '{boardCase}'. 
                You need to select the next team member to speak and suggest a plan or next steps. 
                Here are the roles and descriptions of the participants: 
                {participants}\n
                Please respond with only the role of the participant you would like to select.
                """;

        public static string Filter(string boardCase) =>
            $"""
                ou are a leader of the hospital tumor board. The case the board is working on is : '{boardCase}'. 
                You have just concluded the discussion and team work. 
                Please summarize the discussion and provide the final artifacts.
                """;
    }

    /// <inheritdoc/>
    public override ValueTask<GroupChatManagerResult<string>> FilterResults(ChatHistory history, CancellationToken cancellationToken = default) =>
        this.GetResponseAsync<string>(history, Prompts.Filter(_boardCase), cancellationToken);

    /// <inheritdoc/>
    public override ValueTask<GroupChatManagerResult<string>> SelectNextAgent(ChatHistory history, GroupChatTeam team, CancellationToken cancellationToken = default) =>
        this.GetResponseAsync<string>(history, Prompts.Selection(_boardCase, team.FormatList()), cancellationToken);

    /// <inheritdoc/>
    public override ValueTask<GroupChatManagerResult<bool>> ShouldRequestUserInput(ChatHistory history, CancellationToken cancellationToken = default) =>
        ValueTask.FromResult(new GroupChatManagerResult<bool>(false) { Reason = "The AI group chat manager does not request user input." });

    /// <inheritdoc/>
    public override async ValueTask<GroupChatManagerResult<bool>> ShouldTerminate(ChatHistory history, CancellationToken cancellationToken = default)
    {
        GroupChatManagerResult<bool> result = await base.ShouldTerminate(history, cancellationToken);
        if (!result.Value)
        {
            result = await this.GetResponseAsync<bool>(history, Prompts.Termination(_boardCase), cancellationToken);
        }
        return result;
    }

    private async ValueTask<GroupChatManagerResult<TValue>> GetResponseAsync<TValue>(ChatHistory history, string prompt, CancellationToken cancellationToken = default)
    {
        OpenAIPromptExecutionSettings executionSettings = new() { ResponseFormat = typeof(GroupChatManagerResult<TValue>) };
        ChatHistory request = [.. history, new ChatMessageContent(AuthorRole.System, prompt)];
        ChatMessageContent response = await _chatCompletion.GetChatMessageContentAsync(request, executionSettings, kernel: null, cancellationToken);
        string responseText = response.ToString();
        
        return
            JsonSerializer.Deserialize<GroupChatManagerResult<TValue>>(responseText) ??
            throw new InvalidOperationException($"Failed to parse response: {responseText}");
    }
}

In [12]:
DisplaySection("CUSTOM ORCHESTRATION - Hospital's tumor board is reviewing a complex cancer case", "🎵");

var oncologistAgent = new ChatCompletionAgent
{
    Name = "OncologistAgent",
    Instructions = 
        """
        You are a medical oncologist participating in a multidisciplinary discussion about a cancer patient's treatment. 
        Your role is to evaluate drug-based treatment options, considering cancer stage, biomarkers, and mutation data. 
        You collaborate with a radiologist and a pathologist. Be open to revising your recommendation based on their input.
        Only suggest treatments after reviewing all available clinical and diagnostic information.

        You are the final caller, the decision comes from you and you decide if more inputs are needed from the others.
        The final call and decision is yours! the result of the discussion should come from you so when you think the discussion is over, say it.
        """,
    Description = "A medical oncologist who suggests and revises treatment plans based on pathology and imaging insights.",
    Kernel = kernel,
    Arguments = new KernelArguments(new PromptExecutionSettings{ FunctionChoiceBehavior = FunctionChoiceBehavior.Auto(),})
};

var pathologistAgent = new ChatCompletionAgent
{
    Name = "PathologistAgent",
    Instructions = 
        """
        You are a clinical pathologist providing molecular and genetic analysis of a patient's tumor. 
        Share insights about mutations, biomarkers, and how they might affect treatment effectiveness. 
        Collaborate with the oncologist and radiologist. Suggest when additional lab tests may be needed.
        The final call and decision is of the oncologist!
        """,
    Description = "A clinical pathologist who interprets molecular and genetic data relevant to treatment decisions.",
    Kernel = kernel,
    Arguments = new KernelArguments(new PromptExecutionSettings{ FunctionChoiceBehavior = FunctionChoiceBehavior.Auto(),})

};

var radiologistAgent = new ChatCompletionAgent
{
    Name = "RadiologistAgent",
    Instructions = 
        """
        You are a radiologist interpreting imaging results for a cancer patient. 
        Point out any concerns such as lymph node involvement or metastasis. 
        Inform how imaging findings affect staging and treatment planning. Collaborate with the oncologist and pathologist.
        The final call and decision is of the oncologist!
        """,
    Description = "A radiologist who provides imaging-based insights and identifies implications for treatment planning.",
    Kernel = kernel,
    Arguments = new KernelArguments(new PromptExecutionSettings{ FunctionChoiceBehavior = FunctionChoiceBehavior.Auto(),})
};

// Create a shared chat history for the group consultation
var tumorBoardHistory = new ChatHistory();


🎵 CUSTOM ORCHESTRATION - Hospital's tumor board is reviewing a complex cancer case


In [13]:
#pragma warning disable SKEXP0110
#pragma warning disable SKEXP0001

using Microsoft.SemanticKernel.Agents.Orchestration;
using Microsoft.SemanticKernel.Agents.Runtime.InProcess;

string input =
    """
    Patient: Female, 55 years old  
    Diagnosis: Invasive ductal carcinoma, grade 3  
    Tumor Size: 2.8 cm  
    Stage: T2N0M0 (early-stage)  
    Hormone Receptors: ER+, PR+, HER2-  
    Ki-67: 35% (high proliferation)  
    Genetic Testing: BRCA-negative  
    Genomic Profile: PIK3CA mutation detected  
    Imaging: Mammogram and MRI confirm localized tumor, but one axillary lymph node appears suspicious (not biopsied yet)  
    Comorbidities: Mild hypertension  

    Please collaborate and determine the optimal treatment plan for this patient. Consider systemic therapies, genetic markers, and imaging findings. 
    Discuss whether further testing is needed, and iterate based on each other's input.
    The final call and decision is of the oncologist.
    """;


var chatCompletionService = kernel.GetRequiredService<IChatCompletionService>();

var tumorBoardOrchestration =
   new GroupChatOrchestration(
       new TumorBoardLeader(input, chatCompletionService),
       
       oncologistAgent, 
       pathologistAgent, 
       radiologistAgent)
   {
    ResponseCallback =  chatMessage => 
       { 
            DisplayResponse(chatMessage.AuthorName, chatMessage.Content);
            tumorBoardHistory.Add(chatMessage);
            return ValueTask.CompletedTask;  
       }
   };
// Start the runtime
InProcessRuntime runtime = new();
await runtime.StartAsync();



OrchestrationResult<string> result = await tumorBoardOrchestration.InvokeAsync(input, runtime);
string text = await result.GetValueAsync();
Console.WriteLine($"\n# RESULT: {text}");

await runtime.RunUntilIdleAsync();

Console.WriteLine("\n\nORCHESTRATION HISTORY");
foreach (ChatMessageContent message in tumorBoardHistory)
{
    DisplayResponse(message.AuthorName, message.Content);
}


🤖 **RadiologistAgent:**
   ### Imaging Analysis and Concerns
From the imaging findings (mammogram and MRI), there are the following initial concerns:
1. **Primary Tumor**: Localized invasive ductal carcinoma measuring 2.8 cm without evidence of direct invasion of adjacent structures or distant metastases. Tumor size qualifies as T2 in the TNM staging system. 

2. **Suspicious Axillary Lymph Node**: A single axillary lymph node appears suspicious on imaging, although it has not been biopsied yet. This creates a concern that the current classification of N0 may need revision. A biopsy is critical to confirm if this lymph node contains metastatic carcinoma, which would upstage the disease to N1 (T2N1M0).

---

### Impact on Staging and Treatment
- If the axillary lymph node is positive for metastasis, the disease will upstage to **Stage IIB (T2N1M0)**. 
- This staging adjustment affects both **prognostication** and potential treatment intensification, especially in deciding between syste